In [ ]:
import pandas as pd
from paquo.projects import QuPathProject

from pathlib import Path
from paquo.projects import QuPathProject
from paquo.images import QuPathImageType
from paquo.classes import QuPathPathClass
from shapely.geometry import Point, Polygon 
import numpy as np
import matplotlib.pyplot as plt
from distutils.dir_util import copy_tree


In [ ]:
ml_result = pd.read_csv('../../ML_Layer_prediction/test_features_with_prediction.csv')
image_name='SLD_0000733.vsi - 20x_01'
pixel_size = 0.3460130331522824


layers_color = {"Layer 1": "#ff0000"
                , "Layer 2":"#00ff00"
                , "Layer 3":"#0000ff"
                , "Layer 2/3":"#ffff00"
                ,  "Layer 4":"#00ffff"
                , "Layer 5":"#f0f0f0"
                , "Layer 6 a":"#0ffff0"
                , "Layer 6 b":"#000000"
               }

In [ ]:
annotations = {}
for layer in ["Layer 1", "Layer 2", "Layer 3", "Layer 2/3",  "Layer 4", "Layer 5", "Layer 6 a", "Layer 6 b"]:
    print(f'{layer}')
    layer_points = ml_result[ml_result['Class'] == layer][['Centroid X µm', 'Centroid Y µm']].to_numpy()
    if layer_points.size > 0:
        layer_points = layer_points / pixel_size
        layer_polygon = Polygon(layer_points)
        layer_polygon = Polygon(layer_polygon.convex_hull)

        x, y = layer_polygon.exterior.xy
        plt.plot(x, y, c="red")
        plt.show()

        annotation = {
            f'{layer}': layer_polygon,

        }
        annotations[layer] = annotation


# 


In [ ]:
copy_tree('./project.bck', 'project')
project_path = './project/QuPath_LayerBoundaries_GroundTruth_20220927.qpproj'
with QuPathProject(project_path, mode='r+') as qp:
    print("readed", qp.name)

    
    
    print(f'---INFO: there are {len(qp.images)}  images in this project')
    
    for image in qp.images:
        if image.image_name.find(image_name) > -1:
            print(f'--- INFO: Found [{image.image_name}] ')
            for name, annotation in annotations.items():
                for name, roi in annotation.items():
                # add the annotations without a class set
                    
                    path_class =QuPathPathClass(name=name, color=layers_color[name])
                    print(f'{name} -> {layers_color[name]}')
                    annotation = image.hierarchy.add_annotation(roi=roi, path_class=path_class)
                    annotation.name = name
                    
            
            